
## LangChain Agent for SQLite Database Interaction (CRUD Operations) with DeepSeek
#
### This notebook demonstrates how to create a LangChain agent that can interact with a SQLite database using the DeepSeek LLM. The agent will be able to:
### - **Query (Read)** data from tables.
### - **Add (Create)** new records to tables.
### - **Update** existing records.
### - **Delete** records from tables.
#
### We will use the `SQLDatabaseToolkit` and `create_sql_agent` function provided by LangChain.

### 1. Setup and Installations
### 2. Import Libraries and Configure Environment
#
### We need to import the required modules and set up our environment, particularly the DeepSeek API key. **It's highly recommended to use environment variables for API keys.**


# First, let's install the necessary libraries, including the DeepSeek integration.

In [1]:

!pip3 install -qU langchain langchain-deepseek langchain-community sqlalchemy python-dotenv



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [3]:

from dotenv import load_dotenv
import os
import sqlite3
from langchain_community.utilities import SQLDatabase
# Import ChatDeepseek instead of ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_community.agent_toolkits import SQLDatabaseToolkit, create_sql_agent
from sqlalchemy import create_engine


### Optional: for loading .env files
### from dotenv import load_dotenv

### --- DeepSeek API Key Configuration ---
### Option 1: Set as environment variable (Recommended)
### Make sure you have DEEPSEEK_API_KEY set in your environment.
### os.environ["DEEPSEEK_API_KEY"] = "YOUR_DEEPSEEK_API_KEY" # Replace with your key if hardcoding (NOT recommended for production)
### Option 2: Or load from a .env file

In [ ]:

load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    print("DeepSeek API Key not found. Please set the DEEPSEEK_API_KEY environment variable or place it in a .env file.")


# Check if the key is available
# It's generally better practice to let the library handle the key finding,
# but this check provides immediate feedback if it's missing.
if not os.getenv("DEEPSEEK_API_KEY"):
    print("Warning: DEEPSEEK_API_KEY environment variable not set. The agent might not function.")
    # You might want to raise an error here or exit depending on your needs
    # raise ValueError("DEEPSEEK_API_KEY not found in environment variables.")



### 3. Database Setup
#
### We'll create a simple SQLite database named `user_database.db` with a `users` table. If the database or table already exists, we'll connect to it.


In [ ]:

DB_FILE = "user_database.db"

# Function to initialize the database and table
def initialize_database(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Create users table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER,
        city TEXT
    )
    ''')

    # Check if table is empty and add some initial data if it is
    cursor.execute("SELECT COUNT(*) FROM users")
    count = cursor.fetchone()[0]
    if count == 0:
        print("Adding initial data to 'users' table...")
        cursor.executemany('''
        INSERT INTO users (name, age, city) VALUES (?, ?, ?)
        ''', [
            ('Alice', 30, 'New York'),
            ('Bob', 25, 'Los Angeles'),
            ('Charlie', 35, 'Chicago')
        ])
        print("Initial data added.")
    else:
        print("'users' table already contains data.")

    conn.commit()
    conn.close()
    print(f"Database '{db_file}' initialized successfully.")

# Initialize the database
initialize_database(DB_FILE)


### 4. Connect LangChain to the Database
#
### We use LangChain's `SQLDatabase` utility to wrap our SQLite database connection. This allows LangChain components to easily interact with it.

In [ ]:


# Create SQLAlchemy engine
engine = create_engine(f"sqlite:///{DB_FILE}")

In [ ]:

# Create LangChain SQLDatabase instance
db = SQLDatabase(engine=engine)

In [ ]:

# You can test the connection and see the schema LangChain detects
print("Detected Table Info:")
print(db.table_info)


### 5. Initialize the LLM and SQL Agent
### We'll use `ChatDeepseek` as our language model and then create the SQL agent using `create_sql_agent`. This agent uses the `SQLDatabaseToolkit`, which provides tools for listing tables, getting schema, and executing/checking SQL queries.

### Initialize the LLM using DeepSeek
### Adjust model_name as needed (e.g., "deepseek-chat", "deepseek-coder")
### Check DeepSeek documentation for available model names.

In [ ]:

llm = None # Initialize llm to None
agent_executor = None # Initialize agent_executor to None


In [ ]:


try:
    # Ensure the API key is available before initializing
    if os.getenv("DEEPSEEK_API_KEY"):
        llm = ChatDeepSeek(
            model="deepseek-chat", # Replace with your desired DeepSeek model
            temperature=0,
            # deepseek_api_key=os.getenv("DEEPSEEK_API_KEY") # The library often picks this up automatically from the env var
        )
        print("DeepSeek LLM initialized successfully.")
    else:
        # Skip LLM initialization if key is missing
        print("Skipping LLM initialization because DEEPSEEK_API_KEY is not set.")

except Exception as e:
    print(f"Error initializing DeepSeek LLM: {e}")
    print("Please ensure your DEEPSEEK_API_KEY is set correctly and the model name is valid.")
    # llm remains None

# Proceed only if LLM initialization was successful
if llm:
    # Create the SQLDatabaseToolkit
    # This toolkit bundles tools for interacting with the SQL database
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)

    # Create the agent executor
    # This agent is specifically designed for SQL interactions
    # Note: 'openai-tools' agent_type might still work if DeepSeek's API is compatible
    # or if LangChain handles the translation. Test to confirm.
    # If issues arise, you might need to explore other agent_types or custom agent creation.
    try:
        agent_executor = create_sql_agent(
            llm=llm,
            toolkit=toolkit,
            verbose=True, # Set to True to see the agent's thought process
            agent_type="openai-tools", # Keep this for now, test compatibility
            # You can add prefixes or suffixes to the prompt if needed
            # prefix="You are an agent designed to interact with a SQL database...",
            # suffix="Remember to double-check your SQL queries before execution."
        )
        print("DeepSeek Agent Executor created successfully.")
    except Exception as e:
        print(f"Error creating SQL Agent: {e}")
        agent_executor = None # Ensure agent_executor is None if creation failed

else:
    print("Agent Executor cannot be created because LLM initialization failed or was skipped.")



### 6. Interacting with the Database using the Agent
#
### Now, let's test the agent's capabilities.
### **Note:** These cells will only work if the `agent_executor` was created successfully in the previous step (i.e., the DeepSeek LLM was initialized).


#### 6.1 Querying Data (Read)

#### Example 1: Simple listing

In [ ]:


if agent_executor:
    try:
        response = agent_executor.invoke({"input": "List all users in the database."})
        print("\nAgent Response:")
        print(response['output'])
    except Exception as e:
        print(f"An error occurred during agent invocation: {e}")
else:
    print("Agent not available. Skipping query.")


In [ ]:

# Example 2: Counting records
if agent_executor:
    try:
        response = agent_executor.invoke({"input": "How many users are there?"})
        print("\nAgent Response:")
        print(response['output'])
    except Exception as e:
        print(f"An error occurred during agent invocation: {e}")
else:
    print("Agent not available. Skipping query.")



In [ ]:

# Example 3: Specific query with condition
if agent_executor:
    try:
        response = agent_executor.invoke({"input": "Show me the details for the user named Bob."})
        print("\nAgent Response:")
        print(response['output'])
    except Exception as e:
        print(f"An error occurred during agent invocation: {e}")
else:
    print("Agent not available. Skipping query.")


In [ ]:

# Example 4: Querying specific columns
if agent_executor:
    try:
        response = agent_executor.invoke({"input": "What are the names and cities of all users?"})
        print("\nAgent Response:")
        print(response['output'])
    except Exception as e:
        print(f"An error occurred during agent invocation: {e}")
else:
    print("Agent not available. Skipping query.")



#### 6.2 Adding Data (Create)
#
#### **Important Note:** Allowing an LLM agent to modify data (INSERT, UPDATE, DELETE) carries risks. Ensure you understand the implications and have appropriate safeguards or confirmations in a real application. The agent might misunderstand requests or generate incorrect SQL.

In [ ]:

# Example 5: Adding a new user
if agent_executor:
    try:
        response = agent_executor.invoke({"input": "Add a new user named 'David' who is 40 years old and lives in 'Houston'."})
        print("\nAgent Response:")
        print(response['output'])

        # Verify the addition by querying
        response = agent_executor.invoke({"input": "List all users."})
        print("\nVerification Query Response:")
        print(response['output'])
    except Exception as e:
        print(f"An error occurred during agent invocation: {e}")
else:
    print("Agent not available. Skipping add operation.")


In [ ]:

# ### 6.3 Updating Data (Update)

# Example 6: Updating an existing user's age
if agent_executor:
    try:
        response = agent_executor.invoke({"input": "Update the age of the user named 'Alice' to 31."})
        print("\nAgent Response:")
        print(response['output'])

        # Verify the update
        response = agent_executor.invoke({"input": "Show the details for Alice."})
        print("\nVerification Query Response:")
        print(response['output'])
    except Exception as e:
        print(f"An error occurred during agent invocation: {e}")
else:
    print("Agent not available. Skipping update operation.")


In [ ]:

# ### 6.4 Deleting Data (Delete)
# Example 7: Deleting a user
if agent_executor:
    try:
        # Let's add a temporary user first to delete them safely
        print("Attempting to add temporary user 'DeleteMe'...")
        add_response = agent_executor.invoke({"input": "Add a user named 'DeleteMe' age 99 city 'Nowhere'."})
        print("Add response:", add_response['output'])

        # Now delete the user
        print("\nAttempting to delete user 'DeleteMe'...")
        delete_response = agent_executor.invoke({"input": "Delete the user named 'DeleteMe'."})
        print("\nAgent Response (Delete):")
        print(delete_response['output'])

        # Verify the deletion
        print("\nVerifying deletion...")
        verify_response = agent_executor.invoke({"input": "List all users."})
        print("\nVerification Query Response:")
        print(verify_response['output'])
    except Exception as e:
        print(f"An error occurred during agent invocation: {e}")
else:
    print("Agent not available. Skipping delete operation.")



### 7. How it Works (Briefly)
#
#### 1.  **Input:** You provide a natural language request (e.g., "List all users").
#### 2.  **Agent Thought Process:** The agent (powered by the DeepSeek LLM and toolkit) analyzes the request.
#### 3.  **Tool Selection:** It determines that it needs to interact with the SQL database.
#### 4.  **SQL Generation:** The LLM generates the appropriate SQL query (e.g., `SELECT * FROM users;`).
#### 5.  **SQL Validation (Optional but Recommended):** The toolkit might include steps to validate the SQL syntax.
#### 6.  **SQL Execution:** The agent executes the SQL query against the connected database (`user_database.db`).
#### 7.  **Result:** The database returns the results of the query (or confirms the success/failure of INSERT/UPDATE/DELETE).
#### 8.  **Response Generation:** The agent formats the result into a natural language response (e.g., "Here are the users: ...").
#
#### Setting `verbose=True` when creating the agent allows you to see these intermediate steps (thought process, generated SQL, tool calls, observations).



### 8. Conclusion
#
### We have successfully created a LangChain agent capable of performing Create, Read, Update, and Delete (CRUD) operations on a SQLite database using natural language commands powered by the DeepSeek LLM. This demonstrates the power of combining LLMs with specific toolkits like the `SQLDatabaseToolkit` for interacting with structured data sources.
#
### Remember to exercise caution when granting agents write permissions (INSERT, UPDATE, DELETE) to your databases in production environments. Also, ensure the chosen LLM and agent configuration are suitable and tested for SQL generation accuracy.